## Synchronous and asynchronous UDF execution

This notebook demonstrates the following features that were introduced in release 0.7.0:

* Execute several UDFs in one pass
* Obtain intermediate results from each merge step by executing UDFs as an iterator
* Execute UDFs asynchronously

Please see example live-plotting.ipynb for the related live plotting feature!

In [1]:
import pprint
import asyncio
import copy

import numpy as np

import libertem.api as lt
from libertem.udf.sum import SumUDF
from libertem.udf.sumsigudf import SumSigUDF

In [2]:
ctx = lt.Context()

In [3]:
ds = ctx.load("auto", path="E:/LargeData/LargeData/ER-C-1/projects/ptycho-4.0/data/live-ssb-paper/Ptycho01/20200518 165148/default.hdr")

In [4]:
udfs = [SumUDF(), SumSigUDF()]

### Synchronous execution, only result

Note that both UDFs are executed in a single pass!

In [5]:
res = ctx.run_udf(dataset=ds, udf=udfs)

The result is a tuple with one entry per UDF:

In [6]:
pprint.pprint(res)

({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>})


The previous API when passing a single UDF is not changed, i.e. it doesn't return a tuple but a single UDF result

In [7]:
res = ctx.run_udf(dataset=ds, udf=udfs[0])

In [8]:
pprint.pprint(res)

{'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>}


### Asynchronous execution, only result

By setting `sync=False`, the result is awaitable:

In [9]:
async_res = ctx.run_udf(dataset=ds, udf=udfs, sync=False)
print("Do something else while UDFs are running in the background")
res = await async_res
print("Finished")

Do something else while UDFs are running in the background
Finished


In [10]:
pprint.pprint(res)

({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>})


Just like in the synchronous case, running a single UDF returns the UDF result directly, not a tuple:

In [11]:
async_res = ctx.run_udf(dataset=ds, udf=udfs[0], sync=False)
print("Do something else while UDF is running in the background")
res = await async_res
print("Finished")

Do something else while UDF is running in the background
Finished


In [12]:
pprint.pprint(res)

{'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>}


### Synchronous execution as an iterator

This returns `UDFResults` objects with attributes `buffers` and `damage`. `buffers` is a tuple with the results per UDF, and `damage` is a `BufferWrapper` with `kind='nav'` and `dtype=bool` that indicates which parts of the navigation space have been merged already.

In [13]:
for res in ctx.run_udf_iter(dataset=ds, udf=udfs):
    print(np.count_nonzero(res.damage.data), "nav positions processed")
    pprint.pprint(res.buffers)

2048 nav positions processed
({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>})
4096 nav positions processed
({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>})
6144 nav positions processed
({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>})
8192 nav positions processed
({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>})
10240 nav positions processed
({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>})
12288 nav positions processed
({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
 {'intensity': <BufferWrapper kind=nav dtyp

### Asynchronous execution as an iterator

This allows several iterators to proceed asynchronously in parallel. This approach is used in the backend for the web GUI of LiberTEM to run several analyses concurrently. It could also be useful to implement live feedback to instrument control from UDF results if the control solution works asynchronously.

Note that the UDFs are copied here so that different instances are executed in parallel. Executing the same UDF instances concurrently can lead to undefined behavior.

In [14]:
async def doit(label, udfs):
    async for res in ctx.run_udf_iter(dataset=ds, udf=udfs, sync=False):
        print(label, np.count_nonzero(res.damage.data), "nav positions processed")
        pprint.pprint((label, res.buffers))
    return res
        
p1 = doit("one", copy.deepcopy(udfs))
p2 = doit("two", copy.deepcopy(udfs))
print("Do something else while UDFs are running in the background")
await asyncio.gather(p1, p2)

Do something else while UDFs are running in the background
one 2048 nav positions processed
('one',
 ({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
  {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>}))
two 2048 nav positions processed
('two',
 ({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
  {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>}))
one 4096 nav positions processed
('one',
 ({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
  {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>}))
two 4096 nav positions processed
('two',
 ({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
  {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>}))
two 6144 nav positions processed
('two',
 ({'intensity': <BufferWrapper kind=sig dtype=float32 extra_shape=()>},
  {'intensity': <BufferWrapper kind=nav dtype=float32 extra_shape=()>}))
two 8192 na